In [1]:
import numpy as np
import sys
import hrr_common.utils as cu
from hrr_cobot_robot.gripper_interfaces.wsg_50_hw_local import gripper_command_handle as gch
import hrr_common as hrr_cm
import hrr_cobot_robot as hrr_rob
import spatialmath as sm
import rospy
from tqdm.notebook import tqdm, trange

### ros setup for the lab

In [2]:
import socket
print("for a local setup, replace the above by:\n\t" + \
      r"%env ROS_MASTER_URI=http://localhost:11311")
%env ROS_MASTER_URI=http://hrrN3511rt2004.lsr.ei.tum.de:11311
print("in case this notebook does not work please check that IP below is identical:\n\t"+
      r"%env " + f"ROS_IP={ socket.gethostbyname(socket. gethostname())}\t\t<=>",end="\t")
%env ROS_IP=129.187.147.174

for a local setup, replace the above by:
	%env ROS_MASTER_URI=http://localhost:11311
env: ROS_MASTER_URI=http://hrrN3511rt2004.lsr.ei.tum.de:11311
in case this notebook does not work please check that IP below is identical:
	%env ROS_IP=129.187.147.174		<=>	env: ROS_IP=129.187.147.174


In [ ]:
rospy.init_node('open_pc_clip_dev')
hrr_rob.load_default_parameters("/hrr_cobot")
cobot = hrr_rob.HrrCobotControl.from_ros()
rospy.set_param("~joint_state_topic_name", "/hrr_cobot/joint_states")
rospy.set_param("~tp5_topic_name", "/hrr_cobot/comau_robot_state_controller/comau_cartesian_data")
comau_status = hrr_rob.ComauRobotState.from_ros()
rospy.sleep(1e-1)
print(comau_status)


## Gripper initialization
Assume cobot and gripper have been launched according to 
```bash
roslaunch hrr_cobot_robot hrr_cobot_hw.launch gripper_type:="wsg_50"     
```
```bash
roslaunch hrr_cobot_robot hrr_wsg_50.launch ns:="hrr_cobot"  
```

Use case: Lever/open pc clip to remove side cover

In [ ]:
gripper = gch.GripperCommandHandle('/hrr_cobot/gripper')
gripper.reset()

#Set desired position of gripper
width = 10 #in mm
speed = 100 #in mm/s
gripper.send_pos(width, speed)

#Set velocity (No need probably) What does it do?
gripper.set_vel(5)

#Reset gripper (Homing and rospy sleep for dt=0.1 default)
gripper.reset()

## specify params

### Given from outside

In [1]:
from geometry_msgs.msg import PoseStamped #For testing purposes
object_center = PoseStamped() #PoseStamped
release_pose = PoseStamped() #PoseStamped
gripper_open_width = 50 #float32 #mm #probably should be hardcoded for the use case
gripper_close_width = 50 #float32 #mm #probably should be hardcoded for the use case

In [4]:
object_center.pose.position


x: 0.0
y: 0.0
z: 0.0

### Hardcoded here

In [20]:
#Defaul closing/opening speed of gripper in mm/s
default_speed = 100

#desired distance from goal-pose to pre-pose
pre_pose_dist = 0.01

### We need vector of approach, goal-pose (start of grasp) and end-pose (end of grasp)

In [ ]:
#If we have vector of approach voa, we can do 
voa = np.array([0,0,1])
oc_x = object_center.pose.position.x
oc_y = object_center.pose.position.y
oc_z = object_center.pose.position.z
goal_pose = cu.calc_goal_pose(voa, np.array([oc_x, oc_y, oc_z]))

### For demos just load them manually 

In [ ]:
#np.save('goal_pose_pcclip',cobot.T_B_E_robot,allow_pickle=True) #saves current pose as goal_pose
goal_pose = np.load('goal_pose_pcclip.npy')
sm.SE3(goal_pose)

In [ ]:
#np.save('goal_pose_simgrasp',cobot.T_B_E_robot,allow_pickle=True) #saves current pose as goal_pose
end_pose = np.load('end_pose_pcclip.npy')
sm.SE3(end_pose)

In [ ]:
#nvec: vector of approach in the phase before the gripper closes, to insert one finger under the clip
nvec = np.array([0,0,0])

## Prepare $\rightarrow$ Pre-Pose

### Assume robot is in free space where gripper can be safely opened

In [ ]:
gripper.send_pos(gripper_open_width, default_speed)

### Compute pre-pose (need to include distance from last link to tip of tool here)

In [21]:
nvec = cu.normalize_vec(nvec) #Make sure nvec is normalized
pre_pose = sm.SE3(nvec* pre_pose_dist) @ sm.SE3(goal_pose)
pre_pose

   0.9948    0.04055   0.09362   0.5503    
   0.03814  -0.9989    0.02746   0.01747   
   0.09463  -0.02374  -0.9952    0.5468    
   0         0         0         1         


In [18]:
sm.SE3(goal_pose)

   0.9948    0.04055   0.09362   0.5503    
   0.03814  -0.9989    0.02746   0.02747   
   0.09463  -0.02374  -0.9952    0.5468    
   0         0         0         1         


### Go to pre-pose using traj planner

In [ ]:
 # Initialize the planner action client.
    
# >> Remeber to run the planner node (rosrun planner planner_server_v2.py) in terminal
# >> 
#rospy.init_node('planner_client')

client1 = actionlib.SimpleActionClient('planner', PlannerAction)
client1.wait_for_server()

# Sending deride end configuration
goal = PlannerGoal()
goal.goal_pose.header.frame_id = "map"
goal.goal_pose.header.stamp = rospy.Time.now()

In [ ]:
pre_pose_quat = cu.homog2quat(pre_pose)
pre_pose_pos = cu.homog2pos(pre_pose)
#position of end_configuration
goal.goal_pose.pose.position.x = pre_pose_pos[0]
goal.goal_pose.pose.position.y = pre_pose_pos[1]
goal.goal_pose.pose.position.z = pre_pose_pos[2]

#orientation of end_configuration
goal.goal_pose.pose.orientation.x = pre_pose_quat.x
goal.goal_pose.pose.orientation.y = pre_pose_quat.y
goal.goal_pose.pose.orientation.z = pre_pose_quat.z
goal.goal_pose.pose.orientation.w = pre_pose_quat.w

### Approach and grasp

In [25]:
cobot.move_to_pose(goal_pose)

In [26]:
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/10000 [00:00<?, ?it/s]

[INFO] [1639842070.869527]: reached goal pose at step 120 / 10000


In [ ]:
gripper.send_pos(close_width, default_speed)

### Move to end-pose (here some wiggling might help, maybe giving sinusiodal velocity commands)

In [ ]:
cobot.move_to_pose(end_pose)

In [ ]:
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()

### Open and move to pre-pose

In [ ]:
cobot.move_to_pose(pre_pose)

In [ ]:
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()

In [ ]:
gripper.send_pos(open_width, default_speed)